In [69]:
#Cargar datos de tablas
values = dict()
values["placa"] = [], []
values["cilindro"] = [], []
values["esfera"] = [], []
bis =[]
f = open("TablasTemp.txt", "r")
for line in f:
    if len(line) > 1 and line[0] != "B":
        bi, l1, c1, l2, c2, l3, c3 = line.split('\t')
        bis.append(float(bi))
        values["placa"][0].append(float(l1))
        values["placa"][1].append(float(c1))
        values["cilindro"][0].append(float(l2))
        values["cilindro"][1].append(float(c2))
        values["esfera"][0].append(float(l3))
        values["esfera"][1].append(float(c3))
f.close()

In [70]:
f = open("j.txt", "r")
data = [], []
ns = []
for line in f:
    if len(line) > 1 and line[0] != "B":
        n, j0, j1 = line.split(' ')
        ns.append(float(n))
        data[0].append(float(j0))
        data[1].append(float(j1))
f.close()

In [71]:
def get_j0_and_j1(n: float) -> float:
    try:
        index = ns.index(n)
        return data[0][index], data[1][index] 
    except ValueError:
        for i in range(len(ns)):
            if ns[i] > n:
                num = ns[i] - ns[i-1]
                den1 = data[1][i] - data[1][i-1]
                den2 = data[0][i] - data[0][i-1]
                delta = ((n - ns[i-1])*den1) / num
                delta2 = ((n - ns[i-1])*den2) / num
                c = delta+data[1][i-1]
                l = delta2+data[0][i-1]
                return l, c

In [72]:
get_j0_and_j1(1.09)

(0.72416, 0.46781)

In [73]:
# funcion que nos dara los landas a partir de la tablas, si no lo encuentra calcula el valor interpolado entre los superior e inferior
def getCAndlanda(bi: float,tipo:str)->tuple:

    # Find the index of the bi value in the bis list
    # This is the index of the corresponding c value
    # if the value is not in the list return the amount of upper and downer values
    try:
        index = bis.index(bi)
        return values[tipo][0][index], values[tipo][1][index]
    except ValueError:
        for i in range(len(bis)):
            if bis[i] > bi:
                num = bis[i] - bis[i-1]
                den1 = values[tipo][1][i] - values[tipo][1][i-1]
                den2 = values[tipo][0][i] - values[tipo][0][i-1]
                delta = ((bi - bis[i-1])*den1) / num
                delta2 = ((bi - bis[i-1])*den2) / num
                c = delta+values[tipo][1][i-1]
                l = delta2+values[tipo][0][i-1]
                return l, c


In [74]:
# Datos del problema
diametro = 10 #cm
espesor = 7 #mm
temperatura_horno = 180 #ºC
temperatura_inicial = 25 # ºC
temperatura_coccion = 98
h = 120 #W/m^2K
difusividad =  1.52e-7 # m^2
Cp = 3132
k = 0.51


In [75]:
# Convertimos a unidades del SI
espesor = espesor*1e-3
diametro = diametro*1e-2

In [76]:
#Para la placa 
L_car_placa = espesor/2
Bi_placa = h*L_car_placa/k
Bi_placa # Bi mayor que 0.1

0.8235294117647058

In [77]:
# Obtenemos de tablas
lambda_placa, c_placa = getCAndlanda(Bi_placa, "placa")
print(lambda_placa, c_placa )

0.799564705882353 1.1037411764705882


In [78]:
#para cilindro
L_car_cilindro = diametro/4
Bi_cilindro= h*L_car_cilindro/k

In [79]:
lambda_cilindro, c_cilindro = getCAndlanda(Bi_cilindro, "cilindro")
print(lambda_cilindro, c_cilindro )

2.042035294117647 1.522664705882353


In [80]:
import numpy as np
from sympy import *

In [81]:
t = symbols('t')
t

t

In [82]:
fo_placa =  difusividad/pow(L_car_placa,2)*t
fo_placa

0.0124081632653061*t

In [83]:
fo_cilindro =  difusividad/pow(L_car_cilindro,2)*t
fo_cilindro

0.0002432*t

In [84]:
parte_placa = c_placa*pow(np.exp(1),-pow(lambda_placa,2)*fo_placa)* np.cos(lambda_placa)

parte_placa

0.769328465363375/2.71828182845905**(0.00793258492013841*t)

In [85]:
#get J0
j0, j1 = get_j0_and_j1(lambda_cilindro)
j0

0.19981377647058832

In [86]:
parte_cilindro = c_cilindro*pow(np.exp(1), -pow(lambda_cilindro,2)*fo_cilindro) * j0
parte_cilindro

0.304249385180831/2.71828182845905**(0.00101412166023707*t)

In [87]:
temperatura_coccion = 178

In [88]:
temperaturas = (temperatura_coccion-temperatura_horno)/(temperatura_inicial - temperatura_horno)

print(temperaturas)

0.012903225806451613


In [90]:
parte_total = parte_cilindro * parte_placa
parte_total

0.234067712588919/2.71828182845905**(0.00894670658037548*t)

In [96]:
eqt = parte_total - temperaturas

In [98]:
try:
    solve([eqt], (t))
except Exception as e:
    print(e)

KeyboardInterrupt: 